In [1]:
import os
import cv2
import bm3d
import pickle
import matplotlib.pyplot as plt
from skimage.util import img_as_ubyte
from tqdm.auto import tqdm
from PIL import Image
import numpy as np
import ssim
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import matlab.engine
eng = matlab.engine.start_matlab()
size = (1024, 1024)

In [2]:
eng.cd(r'FastEMD', nargout=0)
eng.compile_FastEMD(nargout=0)
eng.cd(r'..', nargout=0)
eng.path_file(nargout=0)

In [ ]:
gt = cv2.imread('../SIDD/Data/0001_001_S6_00100_00060_3200_L/GT_SRGB_010.PNG')
gt = cv2.resize(gt, size)
gt_img = Image.fromarray(gt, 'RGB')
gt_norm = gt.astype('float64') / 255.0
print(gt.shape, gt.dtype, np.min(gt), np.max(gt))
plt.imshow(gt)
plt.axis('off')

In [ ]:
noisy = cv2.imread('../SIDD/Data/0001_001_S6_00100_00060_3200_L/NOISY_SRGB_010.PNG')
noisy = cv2.resize(noisy, size)
noisy_img = Image.fromarray(noisy, 'RGB')
noisy_norm = noisy.astype('float64') / 255.0
print(noisy.shape, noisy.dtype, np.min(noisy), np.max(noisy))
plt.imshow(noisy)
plt.axis('off')

In [ ]:
noise_psnr = peak_signal_noise_ratio(gt_norm, noisy_norm)
noise_ssim = structural_similarity(gt_norm, noisy_norm, data_range=noisy_norm.max() - noisy_norm.min(), channel_axis=2)
noise_cw = ssim.SSIM(gt_img).cw_ssim_value(noisy_img)
noise_unique = eng.unique_iqa(gt, noisy)
noise_msunique = eng.msunique(gt, noisy)
noise_csv = eng.csv(gt, noisy)
noise_summer = eng.SUMMER(gt, noisy)
print(noise_psnr)   
print(noise_ssim)
print(noise_cw)
print(noise_unique)
print(noise_msunique)
print(noise_csv)
print(noise_summer)

In [3]:
def denoise_nlm(img):
    sigma_est = np.mean(estimate_sigma(img, channel_axis=2))
    denoise_img = denoise_nl_means(img, h= sigma_est, fast_mode=True,
                               patch_size=5, patch_distance=3, channel_axis=2)
    return denoise_img

In [ ]:
# sig = np.mean(estimate_sigma(noisy_norm, channel_axis=2))
# denoised = np.clip(bm3d.bm3d(noisy_norm, sigma_psd=sig, stage_arg=bm3d.BM3DStages.ALL_STAGES), 0.0, 1.0)
denoised = np.clip(denoise_nlm(noisy_norm), 0.0, 1.0)
denoised_img = Image.fromarray(img_as_ubyte(denoised), 'RGB')
noise_psnr = peak_signal_noise_ratio(gt_norm, denoised)
noise_ssim = structural_similarity(gt_norm, denoised, data_range=denoised.max() - denoised.min(), channel_axis=2)
noise_cw = ssim.SSIM(gt_img).cw_ssim_value(denoised_img)
denoised = np.ascontiguousarray(img_as_ubyte(denoised))
noise_unique = eng.unique_iqa(gt, denoised)
noise_msunique = eng.msunique(gt, denoised)
noise_csv = eng.csv(gt, denoised)
noise_summer = eng.SUMMER(gt, denoised)
print(noise_psnr)
print(noise_ssim)
print(noise_cw)
print(noise_unique)
print(noise_msunique)
print(noise_csv)
print(noise_summer)
# plt.imshow(denoised)

In [4]:
def img_dict(dir_path):
    sub_dict = {}
    for item in os.listdir(dir_path):
        item_path = os.path.join(dir_path, item)
        if os.path.isdir(item_path):
            sub_dict[item] = []
    return sub_dict
iqa_file = img_dict('../SIDD/Data')
print(iqa_file)

{'0001_001_S6_00100_00060_3200_L': [], '0002_001_S6_00100_00020_3200_N': [], '0003_001_S6_00100_00060_3200_H': [], '0004_001_S6_00100_00060_4400_L': [], '0005_001_S6_00100_00060_4400_N': [], '0006_001_S6_00100_00060_4400_H': [], '0007_001_S6_00100_00100_5500_L': [], '0008_001_S6_00100_00100_5500_N': [], '0010_001_S6_00800_00350_3200_N': [], '0011_001_S6_00800_00500_5500_L': [], '0012_001_S6_00800_00500_5500_N': [], '0013_001_S6_03200_01250_3200_L': [], '0014_001_S6_03200_01250_3200_N': [], '0015_001_S6_03200_01600_5500_L': [], '0016_001_S6_03200_01600_5500_N': [], '0017_001_GP_00100_00060_5500_N': [], '0018_001_GP_00100_00160_5500_L': [], '0019_001_GP_00800_00640_5500_L': [], '0020_001_GP_00800_00350_5500_N': [], '0022_001_N6_00100_00060_5500_N': [], '0023_001_N6_00800_00350_5500_N': [], '0025_001_G4_00100_00060_5500_L': [], '0027_001_G4_00800_00350_5500_L': [], '0028_001_IP_00100_00160_5500_N': [], '0029_001_IP_00800_01000_5500_N': [], '0030_001_IP_01600_02000_5500_N': [], '0032_001_I

In [5]:
psnr_diff = []
ssim_diff = []
cw_diff = []
u_diff = []
msu_diff = []
csv_diff = []
summer_diff = []

for img in tqdm(iqa_file.keys()):
    print(img)
    folder_path = os.path.join('../SIDD/Data', img)
    imgs = os.listdir(folder_path)
    for img_path in imgs:
        if img_path.startswith("N"): noisy = cv2.imread(os.path.join(folder_path, img_path))
        elif img_path.startswith("G"): gt = cv2.imread(os.path.join(folder_path, img_path))    
    gt = cv2.resize(gt, size)
    gt_img = Image.fromarray(gt, 'RGB')
    gt_norm = gt.astype('float64') / 255.0
    noisy = cv2.resize(noisy, size)
    noisy_img = Image.fromarray(noisy, 'RGB')
    noisy_norm = noisy.astype('float64') / 255.0
    
    noise_psnr = peak_signal_noise_ratio(gt_norm, noisy_norm)
    noise_ssim = structural_similarity(gt_norm, noisy_norm, data_range=noisy_norm.max() - noisy_norm.min(), channel_axis=2)
    noise_cw = ssim.SSIM(gt_img).cw_ssim_value(noisy_img)
    noise_u = eng.unique_iqa(gt, noisy)
    noise_msu = eng.msunique(gt, noisy)
    noise_csv = eng.csv(gt, noisy)
    noise_summer = eng.SUMMER(gt, noisy)   
    
    # sig = np.mean(estimate_sigma(noisy_norm, channel_axis=2))
    # denoised = np.clip(bm3d.bm3d(noisy_norm, sigma_psd=sig, stage_arg=bm3d.BM3DStages.ALL_STAGES), 0.0, 1.0)
    denoised = np.clip(denoise_nlm(noisy_norm), 0.0, 1.0)
    denoised_img = Image.fromarray(img_as_ubyte(denoised), 'RGB')
    
    denoise_psnr = peak_signal_noise_ratio(gt_norm, denoised)
    denoise_ssim = structural_similarity(gt_norm, denoised, data_range=denoised.max() - denoised.min(), channel_axis=2)
    denoise_cw = ssim.SSIM(gt_img).cw_ssim_value(denoised_img)
    denoised = np.ascontiguousarray(img_as_ubyte(denoised))
    denoise_u = eng.unique_iqa(gt, denoised)
    denoise_msu = eng.msunique(gt, denoised)
    denoise_csv = eng.csv(gt, denoised)
    denoise_summer = eng.SUMMER(gt, denoised)
    
    iqa = {"Noise": (noise_psnr, noise_ssim, noise_cw, noise_u, noise_msu, noise_csv, noise_summer),"Denoise": (denoise_psnr,denoise_ssim, denoise_cw, denoise_u, denoise_msu, denoise_csv, denoise_summer)}
    iqa_file[img].append(iqa)
    
    psnr_diff.append(denoise_psnr - noise_psnr)
    ssim_diff.append(denoise_ssim - noise_ssim)
    cw_diff.append(denoise_cw - noise_cw)
    u_diff.append(denoise_u - noise_u)
    msu_diff.append(denoise_msu - noise_msu)
    csv_diff.append(denoise_csv - noise_csv)
    summer_diff.append(denoise_summer - noise_summer)

  0%|          | 0/160 [00:00<?, ?it/s]

0001_001_S6_00100_00060_3200_L
0002_001_S6_00100_00020_3200_N
0003_001_S6_00100_00060_3200_H
0004_001_S6_00100_00060_4400_L
0005_001_S6_00100_00060_4400_N
0006_001_S6_00100_00060_4400_H
0007_001_S6_00100_00100_5500_L
0008_001_S6_00100_00100_5500_N
0010_001_S6_00800_00350_3200_N
0011_001_S6_00800_00500_5500_L
0012_001_S6_00800_00500_5500_N
0013_001_S6_03200_01250_3200_L
0014_001_S6_03200_01250_3200_N
0015_001_S6_03200_01600_5500_L
0016_001_S6_03200_01600_5500_N
0017_001_GP_00100_00060_5500_N
0018_001_GP_00100_00160_5500_L
0019_001_GP_00800_00640_5500_L
0020_001_GP_00800_00350_5500_N
0022_001_N6_00100_00060_5500_N
0023_001_N6_00800_00350_5500_N
0025_001_G4_00100_00060_5500_L
0027_001_G4_00800_00350_5500_L
0028_001_IP_00100_00160_5500_N
0029_001_IP_00800_01000_5500_N
0030_001_IP_01600_02000_5500_N
0032_001_IP_00800_01000_3200_N
0033_001_IP_00100_00160_3200_N
0034_002_GP_00100_00160_3200_N
0035_002_GP_00800_00350_3200_N
0036_002_GP_06400_03200_3200_N
0038_002_GP_00800_00640_3200_L
0039_002

In [6]:
psnr_diff, ssim_diff, cw_diff, u_diff, msu_diff, csv_diff, summer_diff = np.array(psnr_diff), np.array(ssim_diff), np.array(cw_diff), np.array(u_diff), np.array(msu_diff), np.array(csv_diff), np.array(summer_diff)
min_max_mean = {"Min": [np.min(psnr_diff), np.min(ssim_diff), np.min(cw_diff), np.min(u_diff), np.min(msu_diff), np.min(csv_diff), np.min(summer_diff)], "Max": [np.max(psnr_diff), np.max(ssim_diff), np.max(cw_diff), np.max(u_diff), np.max(msu_diff), np.max(csv_diff), np.max(summer_diff)], "Mean": [np.mean(psnr_diff), np.mean(ssim_diff), np.mean(cw_diff), np.mean(u_diff), np.mean(msu_diff), np.mean(csv_diff), np.mean(summer_diff)]}
print(min_max_mean)

{'Min': [-1.2137465776671412, -0.07438777149413889, -0.025749592388155218, -0.14870487506623287, -0.0765185128913517, -0.0026469845636575284, -0.3110556965853619], 'Max': [9.763427594802703, 0.526521655133773, 0.18409606768689385, 0.38999597596813673, 0.4360678853892042, 0.009714975543774207, 0.17750941809185994], 'Mean': [6.360045133477847, 0.2637453994745628, 0.03993923498533034, 0.1559231298188658, 0.10117662128260259, 0.003572657590724241, 0.05105734511190174]}


In [7]:
print(iqa_file)

{'0001_001_S6_00100_00060_3200_L': [{'Noise': (32.88639000031624, 0.5265334089374129, 0.9045818354331012, 0.7891994299781026, 0.878363368838454, 0.9863059015056296, 4.106359226247424), 'Denoise': (40.28141318923182, 0.868068133988776, 0.9335637210930936, 0.8896870488758346, 0.9321573002619452, 0.989217285009644, 4.201067204837708)}], '0002_001_S6_00100_00020_3200_N': [{'Noise': (34.24916960151072, 0.7562805567274612, 0.9408238176175189, 0.8565822134641325, 0.9084878475011776, 0.9879515040742026, 4.199783356928195), 'Denoise': (39.79920024256816, 0.9405300008031806, 0.9580052459904912, 0.9145518845089595, 0.9418716206378267, 0.9897131532090567, 4.252270564828466)}], '0003_001_S6_00100_00060_3200_H': [{'Noise': (34.04920820998789, 0.8674742121827118, 0.9652923661184291, 0.9053760630016813, 0.9340399644053976, 0.9881381657771721, 4.193613235442538), 'Denoise': (36.67361660059938, 0.9590280700212971, 0.9750491212634194, 0.9403239792098036, 0.9547651819680233, 0.9900039949855023, 4.27512505

In [8]:
with open('IQA_SIDD/iqa_SIDD_nlm.pkl', 'wb') as fp:
    pickle.dump(iqa_file, fp)
    print('saved successfully')

with open('IQA_SIDD/iqa_SIDD_nlm_infer.pkl', 'wb') as fp:
    pickle.dump(min_max_mean, fp)   
    print('saved successfully')

saved successfully
saved successfully


In [9]:
with open('IQA_SIDD/iqa_SIDD_nlm.pkl', 'rb') as fp:
    iqa = pickle.load(fp)
    print(iqa)

{'0001_001_S6_00100_00060_3200_L': [{'Noise': (32.88639000031624, 0.5265334089374129, 0.9045818354331012, 0.7891994299781026, 0.878363368838454, 0.9863059015056296, 4.106359226247424), 'Denoise': (40.28141318923182, 0.868068133988776, 0.9335637210930936, 0.8896870488758346, 0.9321573002619452, 0.989217285009644, 4.201067204837708)}], '0002_001_S6_00100_00020_3200_N': [{'Noise': (34.24916960151072, 0.7562805567274612, 0.9408238176175189, 0.8565822134641325, 0.9084878475011776, 0.9879515040742026, 4.199783356928195), 'Denoise': (39.79920024256816, 0.9405300008031806, 0.9580052459904912, 0.9145518845089595, 0.9418716206378267, 0.9897131532090567, 4.252270564828466)}], '0003_001_S6_00100_00060_3200_H': [{'Noise': (34.04920820998789, 0.8674742121827118, 0.9652923661184291, 0.9053760630016813, 0.9340399644053976, 0.9881381657771721, 4.193613235442538), 'Denoise': (36.67361660059938, 0.9590280700212971, 0.9750491212634194, 0.9403239792098036, 0.9547651819680233, 0.9900039949855023, 4.27512505

In [10]:
with open('IQA_SIDD/iqa_SIDD_nlm_infer.pkl', 'rb') as fp:
  inference = pickle.load(fp)
  print(inference)

{'Min': [-1.2137465776671412, -0.07438777149413889, -0.025749592388155218, -0.14870487506623287, -0.0765185128913517, -0.0026469845636575284, -0.3110556965853619], 'Max': [9.763427594802703, 0.526521655133773, 0.18409606768689385, 0.38999597596813673, 0.4360678853892042, 0.009714975543774207, 0.17750941809185994], 'Mean': [6.360045133477847, 0.2637453994745628, 0.03993923498533034, 0.1559231298188658, 0.10117662128260259, 0.003572657590724241, 0.05105734511190174]}
